In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install ultralytics

In [ ]:
#tracking with Byte_track
#results=model.track(source='/content/drive/MyDrive/15sec_input_720p.mp4',show=True,save=True,tracker='bytetrack.yaml',conf=0.20,iou=0.3)

In [ ]:
#python code using OpenCV and YOLO11 to run Object Trcaking on Video Frames and on Live Webcam feed

import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

#Load the YOLO11 Model
model=YOLO('yolo11n.pt')

cap=cv2.VideoCapture('/content/drive/MyDrive/15sec_input_720p.mp4')

# Process a fixed number of frames (e.g., 100 frames)
max_frames = 100
frame_count = 0

while True:
  ret,frame=cap.read()
  if ret and frame_count < max_frames:
    result=model.track(frame,persist=True)
    annotated_frame=result[0].plot()
    cv2_imshow(annotated_frame)

    frame_count += 1
  else:
    break
cap.release()
results=model.track(source='/content/drive/MyDrive/15sec_input_720p.mp4',show=True,save=True)

In [ ]:
import json
import time
from google.colab import files
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/best.pt')
video_path = '/content/drive/MyDrive/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_video_path = '/content/output_l.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
output_data = []

# Initialize ReID feature storage (simple centroid and size)
reid_features = {}  # {id: (centroid_x, centroid_y, width, height)}
reid_threshold = 25  # Distance threshold for ReID matching

def extract_features(bbox):
    x1, y1, x2, y2 = bbox
    centroid_x = (x1 + x2) / 2
    centroid_y = (y1 + y2) / 2
    width = x2 - x1
    height = y2 - y1
    return (centroid_x, centroid_y, width, height)

def reidentify(prev_id, new_id, prev_bbox, new_bbox):
    prev_feat = extract_features(prev_bbox)
    new_feat = extract_features(new_bbox)
    dist = np.sqrt((prev_feat[0] - new_feat[0])**2 + (prev_feat[1] - new_feat[1])**2)
    size_diff = abs(prev_feat[2] * prev_feat[3] - new_feat[2] * new_feat[3])
    if dist < reid_threshold and size_diff < 100:
        return prev_id
    return new_id

# Process video in real-time simulation
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()
    results = model.track(frame, conf=0.6, iou=0.5, persist=True)
    frame_data = {'frame': int(cap.get(cv2.CAP_PROP_POS_FRAMES)), 'players': []}

    if results[0].boxes.id is not None:
        current_ids = results[0].boxes.id.int().tolist()
        current_boxes = results[0].boxes.xyxy.int().tolist()

        # ReID and merge logic
        for box, id in zip(current_boxes, current_ids):
            x1, y1, x2, y2 = box
            new_feature = extract_features(box)
            matched = False

            for prev_id, prev_feat in reid_features.items():
                if reidentify(prev_id, id, prev_feat[1], box) == prev_id:
                    id = prev_id
                    matched = True
                    break

            if not matched:
                reid_features[id] = (frame_count, new_feature)
            frame_data['players'].append({'id': int(id), 'bbox': [x1, y1, x2, y2]})

    output_data.append(frame_data)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

    # Simulate real-time delay
    elapsed = time.time() - start_time
    if elapsed < 1.0 / fps:
        time.sleep(1.0 / fps - elapsed)

    frame_count += 1
    print(f"Processed frame {frame_count}")

cap.release()
out.release()

# Load existing JSON for merging (if available), otherwise use new data
try:
    with open('/content/player_tracking_merged (2).json', 'r') as f:
        merged_data = json.load(f)
    merged_data.extend(output_data)
    output_data = merged_data
except FileNotFoundError:
    pass

# Update ID merging logic
merged_ids = {}
for i in range(len(output_data) - 1):
    frame1 = output_data[i]['players']
    frame2 = output_data[i + 1]['players']
    for p1 in frame1:
        for p2 in frame2:
            if p1['id'] != p2['id']:
                bbox1 = p1['bbox']
                bbox2 = p2['bbox']
                x1, y1, x2, y2 = bbox1
                x3, y3, x4, y4 = bbox2
                w1, h1 = x2 - x1, y2 - y1
                w2, h2 = x4 - x3, y4 - y3
                if (abs(x1 - x3) < 20 and abs(y1 - y3) < 20 and
                    max(w1, w2) - min(w1, w2) < 10 and max(h1, h2) - min(h1, h2) < 10):
                    merged_ids[p2['id']] = p1['id'] if w1 * h1 > w2 * h2 else p2['id']

# Remove duplicates within the same frame
for frame in output_data:
    seen_ids = {}
    for player in frame['players']:
        id = player['id']
        bbox = player['bbox']
        x1, y1, x2, y2 = bbox
        area = (x2 - x1) * (y2 - y1)
        if id in seen_ids:
            if area > seen_ids[id]['area']:
                frame['players'].remove(seen_ids[id]['player'])
            else:
                frame['players'].remove(player)
                continue
        seen_ids[id] = {'player': player, 'area': area}

# Update IDs in data based on merged_ids
for frame in output_data:
    for player in frame['players']:
        if player['id'] in merged_ids:
            player['id'] = merged_ids[player['id']]

# Save the updated JSON file
with open('/content/player_tracking_merged_updated.json', 'w') as f:
    json.dump(output_data, f, indent=2)

# Download the file
files.download('/content/player_tracking_merged_updated.json')

In [ ]:
import json
import matplotlib.pyplot as plt
from collections import defaultdict

# Load the JSON data
file_path = "/content/player_tracking_merged_updated.json"
with open(file_path, 'r') as f:
    tracking_data = json.load(f)

# Track player movement as center coordinates per frame
trajectories = defaultdict(list)

for frame_data in tracking_data:
    for player in frame_data["players"]:
        pid = player["id"]
        x_min, y_min, x_max, y_max = player["bbox"]
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2
        trajectories[pid].append((center_x, center_y))

# Plotting the trajectories
plt.figure(figsize=(12, 8))
for pid, points in trajectories.items():
    xs, ys = zip(*points)
    plt.plot(xs, ys, marker='o', label=f'Player {pid}')

plt.title("Player Movement Trajectories")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.gca().invert_yaxis()  # Y-axis inverted for video frame alignment
plt.tight_layout()
plt.show()
